In [1]:
import sys
sys.path.insert(0, '/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/US/')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import pickle
import random
from scipy import stats
import plotly.express as px
from sklearn.metrics import mean_squared_error, r2_score

import os
from pipeline_config import *
from utils import *
from typing import Dict, Tuple



/Users/mvilenko/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
Lr

0.01

In [4]:
#Seeds for reproducability:

torch.manual_seed(1)
np.random.seed(2)
random.seed(3)

In [5]:
with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/US/basic/test_predictions.pickle', 'rb') as f:
    prediction_dic = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/pickle_files/bi_directional_us_dataset_dict.pickle', 'rb') as f:
    raw_dataset_dict = pickle.load(f)
    
with open(category_id_to_category_name_path, 'rb') as f:
    category_id_to_name_dict = pickle.load(f)
    
with open(categories_per_indent_path, 'rb') as f:
    categories_per_indent_dict = pickle.load(f)
    
with open(test_dataset_path, 'rb') as f:
    test_dataset_dict = pickle.load(f)

In [6]:
def create_test_dataframe(raw_dataset_dict: dict):
    test_dict = {}
    for key in raw_dataset_dict.keys():
    #for key in ['All items']:
        df = raw_dataset_dict[key][['Category', 'Date', 'Year', 'Indent', 'Inflation t+1']]
        df.dropna(inplace=True)
        df.rename(columns={'Inflation t+1': 'Actual'}, inplace=True)
        target_df = df[df['Year'] > Year]
        test_dict[key] = target_df
    return test_dict

test_dict = create_test_dataframe(raw_dataset_dict)


/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_99079/1267119108.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_99079/1267119108.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'Inflation t+1': 'Actual'}, inplace=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_99079/1267119108.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

In [7]:
def get_df_with_predictions(prediction_dic: dict, dict_of_categories_df:dict) -> dict:
    all_data_dict = {}
    for key in list(prediction_dic.keys()):
        predictions = prediction_dic[key]
        prediction_df =  pd.DataFrame(predictions.flatten().detach().numpy())
        prediction_df.rename(columns = {0: 'Prediction'}, inplace=True)
        dict_of_categories_df[key] = dict_of_categories_df[key].reset_index(drop=True)
        all_data_dict[key] = pd.concat([dict_of_categories_df[key], prediction_df], axis=1)
    return all_data_dict

all_data_test_dict=get_df_with_predictions(prediction_dic, test_dict)

In [8]:
# sanity check
all_categories = list(all_data_test_dict.keys())
len(all_categories)

350

In [9]:
horizon_preds = predict_multiple_horizons(weightspath,  test_dataset_dict, categories=all_categories, horizons=8)

Admission to movies, theaters, and concerts
Bacon, breakfast sausage, and related products
Bread other than white
Canned fruits
Canned vegetables
Checking account and other bank services
Chicken
Cookies
Crackers, bread, and cracker products
Dried beans, peas, and lentils
Frankfurters
Fresh cakes and cupcakes
Fresh fish and seafood
Fresh sweetrolls, coffeecakes, doughnuts
Frozen and refrigerated bakery products, pies, tarts, turnovers
Frozen vegetables
Ham
Intracity mass transit
Lunchmeats
Oranges, including tangerines
Other condiments
Other pork including roasts and picnics
Other uncooked poultry including turkey
Parking fees and tolls
Pork chops
Prepared salads
Processed fish and seafood
Salt and other seasonings and spices
Sauces and gravies
Ship fare
Uncooked beef roasts
Uncooked beef steaks
Uncooked ground beef
Uncooked other beef and veal
Video discs and other media
White bread
Admissions
Airline fare
Apparel services other than laundry and dry cleaning
Apples
Bananas
Beef and vea

In [10]:
def create_ground_truth_dict(test_dict: Dict[str, pd.DataFrame], n: int = 8) -> Dict[str, pd.DataFrame]:
    """
    Takes the test dictionary which includes each category test data DataFrame and creates a new dictionary with the ground truth values for n steps ahead.

    Args:
        test_dict (Dict[str, pd.DataFrame]): The test dictionary.
        n (int): The number of steps ahead to predict.

    Returns:
        Dict[str, pd.DataFrame]: The dictionary with the ground truth values.
    """
    gt_dict = {}
    for category in test_dict.keys():
        category_gt_df = pd.DataFrame(
            columns=[f'y_true_{i}' for i in range(1, n+1)])
        category_df = test_dict[category]
        category_df.reset_index(drop=True, inplace=True)
        for i in range(len(category_df)-n):
            category_gt_df.loc[i] = category_df.iloc[i:i+n, -1].values
        gt_dict[category] = category_gt_df
    return gt_dict

In [11]:
ground_truth = create_ground_truth_dict(test_dataset_dict, 8)

In [12]:
category = 'All items'
i = 0
category_gt_df = ground_truth[category]
model_pred = horizon_preds[category]
model_pred_i = model_pred[i]
model_pred_i = model_pred_i.flatten()[:len(category_gt_df)]
model_pred_i.numpy()
#model_pred_i_df.values.flatten()

array([0.27144378, 0.3298781 , 0.38009268, 0.34208557, 0.5234744 ,
       0.45439065, 0.33486164, 0.30905628, 0.35484463, 0.33990884,
       0.18667784, 0.22729763, 0.5944981 , 0.6126448 , 0.70303303,
       0.61796445, 0.65415245, 0.52638835, 0.22295097, 0.543688  ,
       0.6157763 , 0.69082576, 0.4772809 , 0.56812096], dtype=float32)

In [13]:
category_gt_df[[f'y_true_{i+1}']].values.flatten()

array([ 0.35402175,  0.61823765,  0.76708582,  0.64215932,  0.90078677,
        0.4727149 ,  0.27398935,  0.4115879 ,  0.93889903,  0.77609622,
        0.46935007,  0.61943716,  0.79454239,  1.23311989,  0.33138408,
        0.96908903,  1.31357951, -0.01930244,  0.11812671,  0.38522553,
        0.43744175,  0.09624236, -0.07946874,  0.80982923])

In [14]:
model_pred = horizon_preds[category]
model_pred_i = model_pred[i]
model_pred_i = model_pred_i.flatten()[:len(category_gt_df)]
model_pred_i_arr = model_pred_i.numpy()

stats.pearsonr(model_pred_i_arr,model_pred_i_arr)[0]

1.0

In [15]:
gt_arr = category_gt_df[[f'y_true_{i+1}']].values.flatten()
gt_arr.shape[0]

24

In [16]:
gt_arr

array([ 0.35402175,  0.61823765,  0.76708582,  0.64215932,  0.90078677,
        0.4727149 ,  0.27398935,  0.4115879 ,  0.93889903,  0.77609622,
        0.46935007,  0.61943716,  0.79454239,  1.23311989,  0.33138408,
        0.96908903,  1.31357951, -0.01930244,  0.11812671,  0.38522553,
        0.43744175,  0.09624236, -0.07946874,  0.80982923])

In [17]:
gt_arr

array([ 0.35402175,  0.61823765,  0.76708582,  0.64215932,  0.90078677,
        0.4727149 ,  0.27398935,  0.4115879 ,  0.93889903,  0.77609622,
        0.46935007,  0.61943716,  0.79454239,  1.23311989,  0.33138408,
        0.96908903,  1.31357951, -0.01930244,  0.11812671,  0.38522553,
        0.43744175,  0.09624236, -0.07946874,  0.80982923])

In [18]:
ground_truth['All items']

,y_true_1,y_true_2,y_true_3,y_true_4,y_true_5,y_true_6,y_true_7,y_true_8
0,0.354022,0.618238,0.767086,0.642159,0.900787,0.472715,0.273989,0.411588
1,0.618238,0.767086,0.642159,0.900787,0.472715,0.273989,0.411588,0.938899
2,0.767086,0.642159,0.900787,0.472715,0.273989,0.411588,0.938899,0.776096
3,0.642159,0.900787,0.472715,0.273989,0.411588,0.938899,0.776096,0.469350
4,0.900787,0.472715,0.273989,0.411588,0.938899,0.776096,0.469350,0.619437
5,0.472715,0.273989,0.411588,0.938899,0.776096,0.469350,0.619437,0.794542
6,0.273989,0.411588,0.938899,0.776096,0.469350,0.619437,0.794542,1.233120
7,0.411588,0.938899,0.776096,0.469350,0.619437,0.794542,1.233120,0.331384
8,0.938899,0.776096,0.469350,0.619437,0.794542,1.233120,0.331384,0.969089
9,0.776096,0.469350,0.619437,0.794542,1.233120,0.331384,0.969089,1.313580


In [19]:
def calc_pearson_corr(gt_arr, model_pred_i_arr):
    """
    Calculate the pearson correlation between the ground truth and the predictions.

    Args:
        df_gt (pd.DataFrame): The ground truth dataframe.
        df_pred (pd.DataFrame): The predictions dataframe.
    """
    corr = stats.pearsonr(gt_arr,model_pred_i_arr)[0]
    return corr


def calc_rmse(gt_arr, model_pred_i_arr):
    """
    This function calculates the root mean squared error between the ground truth and the predictions.

    Args:
        df_gt (pd.DataFrame): The ground truth dataframe.
        df_pred (pd.DataFrame): The predictions dataframe.

    Returns:
        list: A list of RMSE values for each horizon.
    """
    mse = mean_squared_error(gt_arr, model_pred_i_arr)
    rmse = np.sqrt(mse)

    return rmse
    

def calc_eval_metirc(eval_metric: str, gt_arr, model_pred_i_arr):
    if eval_metric == 'pearson_corr':
            return calc_pearson_corr(gt_arr, model_pred_i_arr)
    elif eval_metric == 'rmse':
        return calc_rmse(gt_arr, model_pred_i_arr)
    # elif eval_metric == 'distance_corr':
    #     return calc_distnace_corr(df_gt, df_pred)
    else:
        raise ValueError('Invalid eval_metric')

def create_eval_dict(ground_truth, horizon_preds, n=8):
# Evaluation metrics
    eval_metrics = ['pearson_corr', 'rmse']
    # Create a dictionary to store the evaluation results
    evaluation_dict = {}
    test_pred_dict = ground_truth
    for category in test_pred_dict.keys():
    #for category in ['All items']:
        print(f'Category: {category}')
        # Get the ground truth dataframe
        category_gt_df = ground_truth[category]
        # Create a dictionary to store the evaluation results for the category
        evaluation_dict[category] = {}
        # Iterate over evaluation metrics
        for metric in eval_metrics:
            # Create a dataframe to store the evaluation results
            df_metric = pd.DataFrame(
                columns=['category', 'model', metric] + [f'horizon_{i}' for i in range(1, n+1)])
            # Iterate over models
            metric_lst = []
            for i in range(n):
                model_pred = horizon_preds[category]
                model_pred_i = model_pred[i]
                model_pred_i = model_pred_i.flatten()[:len(category_gt_df)]
                model_pred_i_arr = model_pred_i.numpy()
                if model_pred_i.shape[0]>0:
                    gt_arr = category_gt_df[[f'y_true_{i+1}']].values.flatten()
                    # Calculate the evaluation metric
                    eval_metric = calc_eval_metirc(metric, gt_arr, model_pred_i_arr)
                    metric_lst.append(eval_metric)
                    # Store the results in the dataframe
            df_metric = df_metric.append({'category': category,  'model': 'Bi-Directional', metric: metric, **{f'horizon_{i+1}': metric_lst[i] for i in range(n)}}, ignore_index=True)
                # Store the results in the dictionary

            evaluation_dict[category][metric] = df_metric
            
    return evaluation_dict

In [20]:
evaluation_dict= create_eval_dict(ground_truth, horizon_preds, n=8)

Category: Admission to movies, theaters, and concerts
Category: Admissions
Category: Airline fare
Category: Alcoholic beverages
Category: Alcoholic beverages at home
Category: Alcoholic beverages away from home
Category: All items
Category: All items less energy


/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_99079/1624155896.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_metric = df_metric.append({'category': category,  'model': 'Bi-Directional', metric: metric, **{f'horizon_{i+1}': metric_lst[i] for i in range(n)}}, ignore_index=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_99079/1624155896.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_metric = df_metric.append({'category': category,  'model': 'Bi-Directional', metric: metric, **{f'horizon_{i+1}': metric_lst[i] for i in range(n)}}, ignore_index=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_99079/1624155896.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_m

Category: All items less food
Category: All items less food and energy
Category: All items less food and shelter
Category: All items less food, shelter, and energy
Category: All items less food, shelter, energy, and used cars and trucks
Category: All items less medical care
Category: All items less shelter
Category: Apparel
Category: Apparel less footwear
Category: Apparel services other than laundry and dry cleaning


/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_99079/1624155896.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_metric = df_metric.append({'category': category,  'model': 'Bi-Directional', metric: metric, **{f'horizon_{i+1}': metric_lst[i] for i in range(n)}}, ignore_index=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_99079/1624155896.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_metric = df_metric.append({'category': category,  'model': 'Bi-Directional', metric: metric, **{f'horizon_{i+1}': metric_lst[i] for i in range(n)}}, ignore_index=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_99079/1624155896.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_m

Category: Apples
Category: Appliances
Category: Audio equipment
Category: Bacon and related products
Category: Bacon, breakfast sausage, and related products
Category: Bakery products
Category: Bananas
Category: Bedroom furniture
Category: Beef and veal
Category: Beer, ale, and other malt beverages at home
Category: Beer, ale, and other malt beverages away from home


...

/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_99079/1624155896.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_metric = df_metric.append({'category': category,  'model': 'Bi-Directional', metric: metric, **{f'horizon_{i+1}': metric_lst[i] for i in range(n)}}, ignore_index=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_99079/1624155896.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_metric = df_metric.append({'category': category,  'model': 'Bi-Directional', metric: metric, **{f'horizon_{i+1}': metric_lst[i] for i in range(n)}}, ignore_index=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_99079/1624155896.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_m

Category: Uncooked beef steaks
Category: Uncooked ground beef
Category: Uncooked other beef and veal
Category: Utilities and public transportation
Category: Vehicle accessories other than tires
Category: Vehicle parts and equipment other than tires
Category: Veterinarian services
Category: Video and audio
Category: Video and audio products
Category: Video and audio services


/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_99079/1624155896.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_metric = df_metric.append({'category': category,  'model': 'Bi-Directional', metric: metric, **{f'horizon_{i+1}': metric_lst[i] for i in range(n)}}, ignore_index=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_99079/1624155896.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_metric = df_metric.append({'category': category,  'model': 'Bi-Directional', metric: metric, **{f'horizon_{i+1}': metric_lst[i] for i in range(n)}}, ignore_index=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_99079/1624155896.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_m

Category: Video discs and other media
Category: Watches
Category: Water and sewer and trash collection services
Category: Water and sewerage maintenance
Category: Whiskey at home
Category: White bread
Category: Window and floor coverings and other linens
Category: Window coverings
Category: Wine at home
Category: Wine away from home
Category: Wireless telephone services
Category: Women’s and girls’ apparel
Category: Women’s apparel
Category: Women’s dresses
Category: Women’s footwear
Category: Women’s outerwear
Category: Women’s suits and separates
Category: Women’s underwear, nightwear, sportswear and accessories


/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_99079/1624155896.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_metric = df_metric.append({'category': category,  'model': 'Bi-Directional', metric: metric, **{f'horizon_{i+1}': metric_lst[i] for i in range(n)}}, ignore_index=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_99079/1624155896.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_metric = df_metric.append({'category': category,  'model': 'Bi-Directional', metric: metric, **{f'horizon_{i+1}': metric_lst[i] for i in range(n)}}, ignore_index=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_99079/1624155896.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_m

In [21]:
evaluation_dict['All items']['pearson_corr']

,category,model,pearson_corr,horizon_1,horizon_2,horizon_3,horizon_4,horizon_5,horizon_6,horizon_7,horizon_8
0,All items,Bi-Directional,pearson_corr,0.195176,-0.336154,0.000757,0.054209,-0.381515,-0.299502,-0.219855,-0.146985


In [22]:
evaluation_dict['All items']['rmse']['horizon_1'].values[0]

0.37972676431008373

In [23]:
def create_avg_eval(evaluation_dict):
    rmse_dict = {}
    corr_dict = {}

    for i in range(8):
        rmse_dict[f'horizon_{i+1}'] = []
        corr_dict[f'horizon_{i+1}'] = []

    for category in list(evaluation_dict.keys()):
        for i in range(8):
            rmse_dict[f'horizon_{i+1}'].append(evaluation_dict[category]['rmse'][f'horizon_{i+1}'].values[0])
            corr_dict[f'horizon_{i+1}'].append(evaluation_dict[category]['pearson_corr'][f'horizon_{i+1}'].values[0])

    df_metric = pd.DataFrame(
                    columns=['metric'] + [f'horizon_{i}' for i in range(1, 9)])

    rmse_lst = []
    corr_lst = []
    for i in range(8):
        rmse_lst.append(sum(rmse_dict[f'horizon_{i+1}'])/len(rmse_dict[f'horizon_{i+1}']))
        corr_lst.append(sum(corr_dict[f'horizon_{i+1}'])/len(corr_dict[f'horizon_{i+1}']))

    df_metric = df_metric.append({'metric': 'rmse', **{f'horizon_{i+1}': rmse_lst[i] for i in range(8)}}, ignore_index=True) 
    df_metric = df_metric.append({'metric': 'pearson_corr', **{f'horizon_{i+1}': corr_lst[i] for i in range(8)}}, ignore_index=True) 

    return df_metric

In [24]:
horizon_eval = create_avg_eval(evaluation_dict)

/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_99079/2884383479.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_metric = df_metric.append({'metric': 'rmse', **{f'horizon_{i+1}': rmse_lst[i] for i in range(8)}}, ignore_index=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_99079/2884383479.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_metric = df_metric.append({'metric': 'pearson_corr', **{f'horizon_{i+1}': corr_lst[i] for i in range(8)}}, ignore_index=True)


In [25]:
horizon_eval

,metric,horizon_1,horizon_2,horizon_3,horizon_4,horizon_5,horizon_6,horizon_7,horizon_8
0,rmse,1.581728,1.705676,1.703240,1.717173,1.758080,1.757298,1.765496,1.776739
1,pearson_corr,0.127837,0.014361,0.023517,0.007814,-0.000813,0.003920,-0.003232,-0.026090


Average RMSE:

In [26]:
def avg_rmse(all_data_test_dict):
    mse_lst = []
    for key in all_categories:
        df_predictions = all_data_test_dict[key]
        y_pred = df_predictions['Prediction'].values
        y_actual = df_predictions['Actual'].values
        mse = mean_squared_error(y_pred, y_actual)
        mse_lst.append(mse)
    
    rmse_list = list(map(np.sqrt,mse_lst))
    avg_rmse = np.mean(rmse_list)
    rmse_std = np.std(rmse_list)
    
    print(f'RMSE:  {avg_rmse}')
    print(f'MSE std:  {rmse_std}')
    print(f'interval: {[avg_rmse-rmse_std, avg_rmse+rmse_std]}')

    return avg_rmse,rmse_std

avg_rmse(all_data_test_dict)

RMSE:  1.5330139184906966
MSE std:  1.1477380685127423
interval: [0.3852758499779543, 2.680751987003439]


(1.5330139184906966, 1.1477380685127423)

Headline RMSE:

In [27]:
df_predictions = all_data_test_dict['All items']
y_pred = df_predictions['Prediction'].values
y_actual = df_predictions['Actual'].values
mse = mean_squared_error(y_pred, y_actual)
rmse = np.sqrt(mse)


print(f'rmse: {rmse}')

rmse: 0.3575319622020888


In [28]:
print(f'y_pred: {y_pred}')
print(f'y_actual: {y_actual}')

y_pred: [0.27144372 0.3298781  0.3800927  0.34208563 0.5234744  0.4543907
 0.33486164 0.30905625 0.35484463 0.3399087  0.18667789 0.22729772
 0.59449804 0.6126448  0.70303303 0.61796445 0.6541525  0.5263883
 0.22295079 0.54368794 0.61577624 0.69082576 0.47728086 0.5681209
 0.52996725 0.6486745  0.571526   0.45804924 0.23348895 0.15624402
 0.33847946 0.48714274]
y_actual: [ 0.35402175  0.61823765  0.76708582  0.64215932  0.90078677  0.4727149
  0.27398935  0.4115879   0.93889903  0.77609622  0.46935007  0.61943716
  0.79454239  1.23311989  0.33138408  0.96908903  1.31357951 -0.01930244
  0.11812671  0.38522553  0.43744175  0.09624236 -0.07946874  0.80982923
  0.36932275  0.05302789  0.36710882  0.12404903  0.18019062  0.16672453
  0.62920172  0.3949496 ]


Total Correlation:

In [29]:
def total_corr(all_data_test_dict):
    corr_dict = {}
    for key in all_categories:
        df_predictions = all_data_test_dict[key]
        y_pred = df_predictions['Prediction'].values
        y_actual = df_predictions['Actual'].values
        corr = stats.pearsonr(y_pred,y_actual)[0]
        corr_dict[key] =  corr
    
    total_corr = sum(corr_dict.values())
    
    num_high_corr = 0
    for category in corr_dict:
        if corr_dict[category] >= 0.5:
            num_high_corr +=1
    
    print(f'Number of categories with High Correlation: {num_high_corr}')
    
    return total_corr

total_corr(all_data_test_dict)

Number of categories with High Correlation: 14


49.50597518590682

In [30]:
def total_corr_all_items(all_data_test_dict):
    corr_dict = {}
    for key in ['All items']:
        df_predictions = all_data_test_dict[key]
        y_pred = df_predictions['Prediction'].values
        y_actual = df_predictions['Actual'].values
        corr = stats.pearsonr(y_pred,y_actual)[0]
        corr_dict[key] =  corr
    
    total_corr = sum(corr_dict.values())
    
    num_high_corr = 0
    for category in corr_dict:
        if corr_dict[category] >= 0.5:
            num_high_corr +=1
    
    print(f'Number of categories with High Correlation: {num_high_corr}')
    
    return total_corr

total_corr_all_items(all_data_test_dict)

Number of categories with High Correlation: 0


0.17142569563418392

Average R2:

In [31]:
def avg_r_squared(all_data_test_dict):
    r_squared_lst = []

    for key in all_categories:
        #print(f'category: {key}')
        df_predictions = all_data_test_dict[key]
        y_pred = df_predictions['Prediction'].values
        y_actual = df_predictions['Actual'].values
        r2 = r2_score(y_actual, y_pred)
        r_squared_lst.append(r2)

        if key =='All items':
            headline_r2 = r2
    
    avg_r2_score = np.mean(r_squared_lst)
    r2_std = np.std(r_squared_lst)
    
    print(f'Average R Squared:  {avg_r2_score}')
    print(f'R Squared std:  {r2_std}')
    print('--------------------------------------------------------')
    print(f'Headline R2: {headline_r2}')
    print(f'R2 list percentiles:\n[10: {np.percentile(r_squared_lst, 10)}, 25: {np.percentile(r_squared_lst, 25)}, 50: {np.percentile(r_squared_lst, 50)}, 75: {np.percentile(r_squared_lst, 75)}, 90: {np.percentile(r_squared_lst, 90)}]')

    return headline_r2, avg_r2_score

headline_r2, avg_r2_score = avg_r_squared(all_data_test_dict)

Average R Squared:  -0.8945105658906248
R Squared std:  2.384741600863208
--------------------------------------------------------
Headline R2: -0.07365855556097323
R2 list percentiles:
[10: -1.6338780346226707, 25: -0.8373074235134559, 50: -0.4443934836043101, 75: -0.15326587440293332, 90: 0.03520747177557882]


Plot Results:

In [32]:
def plot_results(all_data_dict, categories):
    category_samples = ['All items']+random.sample(categories, 10)
    for category in category_samples:
        category_df = all_data_dict[category]
        fig = px.line(category_df, x="Date", y=["Actual", "Prediction"], title=f'{category} - Actual VS Prediction')
        fig.show()

plot_results(all_data_test_dict, all_categories)